# 卷积神经网络

In [62]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = False

train_data = torchvision.datasets.MNIST(
    root = './mnist',
    train = True,
    transform = torchvision.transforms.ToTensor(),  #黑白图片的像素值从（0,255）压缩为(0,1)
    download = DOWNLOAD_MNIST
)

# print(train_data.train_data.size())
# print(train_data.train_labels.size())
# plt.imshow(train_data.train_data[0].numpy(),cmap = 'gray')
# plt.title('%i'%train_data.train_labels[0])
# plt.show()

train_loader = Data.DataLoader(
    dataset = train_data,
    batch_size=BATCH_SIZE,
    shuffle = True,
    num_workers=2
)

test_data = torchvision.datasets.MNIST(root='./mnist/',train=False)

test_x = Variable(torch.unsqueeze(test_data.test_data,dim=1)).type(torch.FloatTensor)[:2000]/255
test_y = test_data.test_labels[:2000]

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d( # (1,28,28)
                in_channels = 1,
                out_channels = 16, #(16,28,28)
                kernel_size = 5,
                stride = 1,
                padding = 2, 
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #(16,14,14)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels = 16, # (16,14,14)
                out_channels = 32, #(32,14,14)
                kernel_size = 5,
                stride = 1,
                padding = 2, 
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), #(32,7,7)
        )
        
        self.out = nn.Linear(32*7*7,10)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)  # (batch,32,7,7)
        x = x.view(x.size(0),-1)  # (batch,32*7*7)
        output = self.out(x)
        return output

cnn = CNN()
optimizer = torch.optim.Adam(cnn.parameters(),lr=LR)
loss_func = nn.CrossEntropyLoss()

    
for epoch in range(EPOCH):
    for step,(x,y) in enumerate(train_loader):
        b_x = Variable(x)
        b_y = Variable(y)
        output = cnn(b_x)
        loss = loss_func(output,b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 ==0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output,1)[1].data.squeeze()
            accuracy = (pred_y==test_y).sum().item()/test_y.size(0)
            print("Epoch:",epoch,"|train loss: %.4f"%loss,"|test accuracy:",accuracy)
            
test_output = cnn(test_x[-15:])
pred_y = torch.max(test_output,1)[1].data.numpy().squeeze()
print(pred_y,"prediction number")
print(test_y[-15:].numpy(),'real number')


Epoch: 0 |train loss: 2.3011 |test accuracy: 0.172
Epoch: 0 |train loss: 0.5966 |test accuracy: 0.7975
Epoch: 0 |train loss: 0.5038 |test accuracy: 0.882
Epoch: 0 |train loss: 0.4106 |test accuracy: 0.9115
Epoch: 0 |train loss: 0.1879 |test accuracy: 0.9285
Epoch: 0 |train loss: 0.0491 |test accuracy: 0.9175
Epoch: 0 |train loss: 0.2059 |test accuracy: 0.944
Epoch: 0 |train loss: 0.0984 |test accuracy: 0.9515
Epoch: 0 |train loss: 0.3002 |test accuracy: 0.954
Epoch: 0 |train loss: 0.1004 |test accuracy: 0.9545
Epoch: 0 |train loss: 0.1390 |test accuracy: 0.962
Epoch: 0 |train loss: 0.1288 |test accuracy: 0.964
Epoch: 0 |train loss: 0.0315 |test accuracy: 0.97
Epoch: 0 |train loss: 0.1200 |test accuracy: 0.969
Epoch: 0 |train loss: 0.0812 |test accuracy: 0.975
Epoch: 0 |train loss: 0.1301 |test accuracy: 0.9705
Epoch: 0 |train loss: 0.2323 |test accuracy: 0.9745
Epoch: 0 |train loss: 0.0257 |test accuracy: 0.969
Epoch: 0 |train loss: 0.0868 |test accuracy: 0.9745
Epoch: 0 |train loss: 0